In [155]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier

In [156]:
ddf = pd.read_csv('test2k.csv',na_values='?')
ddf.head()

,name,cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,CC1=CC(=C(N1C)C)C2=CSC(=N2)N,1,NaN,NaN,0,0,17.701129,2.439857,4.640601,17.701129,...,9.512739,61.847126,207.083018,7.669741,288,19,74,88,5.666667,3.027778
1,CCCOC1=CC=C(C=C1)C(=O)N/N=C/C2=CC=C(O2)C3=CC=C...,1,NaN,NaN,0,0,46.736633,2.407624,4.793640,46.736633,...,10.344288,85.856674,505.141990,8.561729,5352,52,184,211,10.701389,8.013889
2,COC1=CC(=C(C=C1)C(C[N+](=O)[O-])C2=C(C=C3N2C=C...,1,NaN,NaN,0,0,39.586705,2.537049,4.973884,39.586705,...,10.368070,80.145118,402.157957,7.733807,2195,50,158,189,9.111111,6.777778
3,CC1=C(C(=O)N(N1)C2=CC=CC=C2)N=NC3=NC=CS3,1,NaN,NaN,0,0,26.395318,2.442924,4.716386,26.395318,...,9.777074,69.229338,285.068431,9.195756,853,27,104,122,5.666667,4.444444
4,CCCCN=C1N(C(=CS1)C2=CC3=C(C=C2)OCC(=O)N3)N=C4C...,1,NaN,NaN,0,0,35.597476,2.464844,4.867887,35.597476,...,10.147649,77.687460,384.161997,7.532588,1886,39,142,167,7.138889,6.027778


In [157]:
ddf.dropna(how='all', axis=1, inplace=True)

In [158]:
ddf.head()

,name,cls,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,CC1=CC(=C(N1C)C)C2=CSC(=N2)N,1,0,0,17.701129,2.439857,4.640601,17.701129,1.264366,3.582199,...,9.512739,61.847126,207.083018,7.669741,288,19,74,88,5.666667,3.027778
1,CCCOC1=CC=C(C=C1)C(=O)N/N=C/C2=CC=C(O2)C3=CC=C...,1,0,0,46.736633,2.407624,4.793640,46.736633,1.298240,4.501393,...,10.344288,85.856674,505.141990,8.561729,5352,52,184,211,10.701389,8.013889
2,COC1=CC(=C(C=C1)C(C[N+](=O)[O-])C2=C(C=C3N2C=C...,1,0,0,39.586705,2.537049,4.973884,39.586705,1.319557,4.334574,...,10.368070,80.145118,402.157957,7.733807,2195,50,158,189,9.111111,6.777778
3,CC1=C(C(=O)N(N1)C2=CC=CC=C2)N=NC3=NC=CS3,1,0,0,26.395318,2.442924,4.716386,26.395318,1.319766,3.934751,...,9.777074,69.229338,285.068431,9.195756,853,27,104,122,5.666667,4.444444
4,CCCCN=C1N(C(=CS1)C2=CC3=C(C=C2)OCC(=O)N3)N=C4C...,1,0,0,35.597476,2.464844,4.867887,35.597476,1.318425,4.235035,...,10.147649,77.687460,384.161997,7.532588,1886,39,142,167,7.138889,6.027778


In [159]:
ddf.shape

(2401, 1517)

In [161]:
df = ddf[[c for c in ddf if c not in ['cls']] 
       + ['cls']]

In [162]:
df.head()

,name,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,cls
0,CC1=CC(=C(N1C)C)C2=CSC(=N2)N,0,0,17.701129,2.439857,4.640601,17.701129,1.264366,3.582199,3.443387,...,61.847126,207.083018,7.669741,288,19,74,88,5.666667,3.027778,1
1,CCCOC1=CC=C(C=C1)C(=O)N/N=C/C2=CC=C(O2)C3=CC=C...,0,0,46.736633,2.407624,4.793640,46.736633,1.298240,4.501393,4.490651,...,85.856674,505.141990,8.561729,5352,52,184,211,10.701389,8.013889,1
2,COC1=CC(=C(C=C1)C(C[N+](=O)[O-])C2=C(C=C3N2C=C...,0,0,39.586705,2.537049,4.973884,39.586705,1.319557,4.334574,4.469321,...,80.145118,402.157957,7.733807,2195,50,158,189,9.111111,6.777778,1
3,CC1=C(C(=O)N(N1)C2=CC=CC=C2)N=NC3=NC=CS3,0,0,26.395318,2.442924,4.716386,26.395318,1.319766,3.934751,3.715307,...,69.229338,285.068431,9.195756,853,27,104,122,5.666667,4.444444,1
4,CCCCN=C1N(C(=CS1)C2=CC3=C(C=C2)OCC(=O)N3)N=C4C...,0,0,35.597476,2.464844,4.867887,35.597476,1.318425,4.235035,4.417538,...,77.687460,384.161997,7.532588,1886,39,142,167,7.138889,6.027778,1


In [163]:
X, y = df.iloc[:, :-1], df.iloc[:,-1]

In [164]:
y.value_counts()

cls
0    1201
1    1200
Name: count, dtype: int64

In [165]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: cls, dtype: int64

Null values

In [166]:
X.isna().sum().sum()

193057

In [230]:
X1, X2, y1, y2 = train_test_split(X, y, random_state=0, test_size=0.1)

In [231]:
y2.head()

1951    0
252     1
2303    0
1665    0
398     1
Name: cls, dtype: int64

In [232]:
X1.head()

,name,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
137,C1=CC=C2C(=C1)C(=O)C(=CC3=CC=CS3)C2=O,0,0,22.682931,2.493840,4.740819,22.682931,1.334290,3.791557,3.579368,...,9.797516,66.082545,240.024500,9.600980,494,25,92,111,4.916667,3.722222
838,C1CN(CCC1N2C(=CC=N2)NC(=O)C3CCOC3)CC4=CC5=CC=C...,0,1,38.962809,2.415827,4.740745,38.962809,1.343545,4.323239,4.958154,...,10.222923,81.386497,393.216475,7.021723,2597,41,158,187,6.750000,6.277778
2035,CC1CCCN(C1)C(=O)CN2C3=CC=CC=C3OC2=O,0,0,25.877322,2.453323,4.770194,25.877322,1.293866,3.937337,3.752670,...,9.899228,67.995365,274.131742,7.213993,815,30,106,125,6.277778,4.361111
1912,COC1=CC=CC=C1CNC(=O)COC(=O)C2=NC=CN=C2,0,0,28.689140,2.273115,4.546230,28.689140,1.304052,3.985056,4.121461,...,9.622913,55.161329,301.106256,8.138007,1267,29,104,116,7.055556,5.138889
1456,C1=CC=C2C(=C1)N=CN2CC3=NN=C4N3N=C(S4)C5=CC=NC=C5,0,0,32.996057,2.494952,4.775003,32.996057,1.374836,4.156422,4.321836,...,10.178350,76.202288,333.079664,9.516562,1392,36,136,165,4.888889,5.138889


In [233]:
knn = KNNImputer()
f1 = X1.loc[:, X1.columns != 'name']
f2 = X2.loc[:, X2.columns != 'name']
knn.fit(f1)

X1_t = knn.transform(f1)
X2_t = knn.transform(f2)

In [234]:
mm = MinMaxScaler()
X1_t = mm.fit_transform(X1_t)
X2_t = mm.transform(X2_t)

In [235]:
vt = VarianceThreshold(threshold = 0.03)
vt.fit_transform(X1_t).shape

(2160, 85)

In [236]:
X1_t.shape

(2160, 1513)

In [237]:
vt.get_support()

array([False,  True, False, ..., False, False, False])

In [238]:
np.var(X1_t[:, 2])

0.008878772355645045

In [239]:
vt.transform(X2_t).shape

(241, 85)

In [240]:
f1.head()

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
137,0,0,22.682931,2.493840,4.740819,22.682931,1.334290,3.791557,3.579368,0.210551,...,9.797516,66.082545,240.024500,9.600980,494,25,92,111,4.916667,3.722222
838,0,1,38.962809,2.415827,4.740745,38.962809,1.343545,4.323239,4.958154,0.170971,...,10.222923,81.386497,393.216475,7.021723,2597,41,158,187,6.750000,6.277778
2035,0,0,25.877322,2.453323,4.770194,25.877322,1.293866,3.937337,3.752670,0.187634,...,9.899228,67.995365,274.131742,7.213993,815,30,106,125,6.277778,4.361111
1912,0,0,28.689140,2.273115,4.546230,28.689140,1.304052,3.985056,4.121461,0.187339,...,9.622913,55.161329,301.106256,8.138007,1267,29,104,116,7.055556,5.138889
1456,0,0,32.996057,2.494952,4.775003,32.996057,1.374836,4.156422,4.321836,0.180076,...,10.178350,76.202288,333.079664,9.516562,1392,36,136,165,4.888889,5.138889


In [241]:
pipe_wo_vt = Pipeline([('s1', KNNImputer()),('s2', MinMaxScaler()),#('s3', VarianceThreshold(threshold=0.01)),
                       ('s4', RandomForestClassifier(random_state=0))])

In [242]:
y1.head()

137     1
838     1
2035    0
1912    0
1456    0
Name: cls, dtype: int64

In [243]:
scores_wo_vt = cross_val_score(pipe_wo_vt, f1, y1, n_jobs=-1, cv=StratifiedKFold(random_state=0, shuffle=True))

In [244]:
scores_wo_vt.mean()

0.7847222222222222

In [245]:
pipe_with_vt = Pipeline([('s1', KNNImputer()),('s2', MinMaxScaler()),('s3', VarianceThreshold(threshold=0.03)),
                       ('s4', RandomForestClassifier(random_state=0))])

In [246]:
scores_with_vt = cross_val_score(pipe_with_vt, f1, y1, n_jobs=-1, cv=StratifiedKFold(random_state=0, shuffle=True))

In [247]:
scores_with_vt.mean()

0.7967592592592593